In [1]:

import sqlite3

import pandas

from data_algebra.data_ops import *
import data_algebra.SQLite

db_model = data_algebra.SQLite.SQLiteModel()

# some example data
d = pandas.DataFrame({
    'group': ['A', 'B', None, 'A', 'B', 'C'],
    'c1': [1, 2, 3, 4, 5, 6],
    'c2': [-1, -2, -3, -4, -5, -6],
    })

d

,group,c1,c2
0,A,1,-1
1,B,2,-2
2,None,3,-3
3,A,4,-4
4,B,5,-5
5,C,6,-6


In [2]:
ops = describe_table(d, table_name='d'). \
    extend({'choice': "group=='A'"}). \
    extend({'rc': 'choice.if_else(c1, c2)'})

print(ops)

TableDescription(
 table_name='d',
 column_names=[
   'group', 'c1', 'c2']) .\
   extend({
    'choice': "group == 'A'"}) .\
   extend({
    'rc': 'choice.if_else(c1, c2)'})


In [3]:
ops.transform(d)



,group,c1,c2,choice,rc
0,A,1,-1,True,1
1,B,2,-2,False,-2
2,None,3,-3,False,-3
3,A,4,-4,True,4
4,B,5,-5,False,-5
5,C,6,-6,False,-6
